In [22]:
%load_ext autoreload
%autoreload 2
import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from sklearn.decomposition import PCA
from linear_probe import get_linear_acc
import random
import os
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve,f1_score
from ml_utils.metrics import aur_pr
from answer_judge import load_judge

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
data_train=ReadData('/root/autodl-fs/hallscope/ScienceQA/train').read_all()
data_val=ReadData('/root/autodl-fs/hallscope/ScienceQA/val').read_all()
data_test=ReadData('/root/autodl-fs/hallscope/ScienceQA/test').read_all()

In [3]:
df_train=pd.DataFrame(data_train)
df_val=pd.DataFrame(data_val)
df_test=pd.DataFrame(data_test)

In [4]:
df_train.head()

,question,answer,most_likely
0,[Question:Which of these states is farthest no...,"[tensor(0), tensor(1), tensor(0), tensor(1), t...",{'embedding': [[[ 7.2813034e-04 6.3419342e-04...
1,[Question:What do these two changes have in co...,"[tensor(0), tensor(0), tensor(0), tensor(2), t...",{'embedding': [[[ 7.2813034e-04 6.3419342e-04...
2,[Question:Which property matches this object?\...,"[tensor(0), tensor(0), tensor(1), tensor(0), t...",{'embedding': [[[ 7.2813034e-04 6.3419342e-04...
3,[Question:What information supports the conclu...,"[tensor(1), tensor(1), tensor(1), tensor(1), t...",{'embedding': [[[ 7.2813034e-04 6.3419342e-04...
4,[Question:What is the expected ratio of offspr...,"[tensor(1), tensor(0), tensor(1), tensor(0), t...",{'embedding': [[[ 7.2813034e-04 6.3419342e-04...


In [27]:
i=0
df_train.iloc[i]['most_likely']['response']

['A',
 'B. Do ping pong balls travel farther when launched from a 30° angle compared to a 45° angle?',
 "A. Does Kathleen's snowboard slide down a hill in less time when it has a layer of wax or when it does not have a layer of wax?",
 'C. past tense',
 'B',
 'B',
 'C.4/4',
 'A.neither; the samples have the same temperature']

In [28]:
df_train.iloc[i]['question']

['Question:Which of these states is farthest north?\nAnswer the Question with following choices.\nChoices: A.West Virginia B.Louisiana C.Arizona D.Oklahoma\n',
 "Question:Identify the question that Tom and Justin's experiment can best answer.\nAnswer the Question with following choices.\nChoices: A.Do ping pong balls stop rolling along the ground sooner after being launched from a 30° angle or a 45° angle? B.Do ping pong balls travel farther when launched from a 30° angle compared to a 45° angle?\n",
 "Question:Identify the question that Kathleen and Bryant's experiment can best answer.\nAnswer the Question with following choices.\nChoices: A.Does Kathleen's snowboard slide down a hill in less time when it has a layer of wax or when it does not have a layer of wax? B.Does Kathleen's snowboard slide down a hill in less time when it has a thin layer of wax or a thick layer of wax?\n",
 'Question:Which tense does the sentence use?\nMona will print her name with care.\nAnswer the Question 

In [29]:
embedding_train=df_train['most_likely'].apply(lambda x: x['embedding']).to_list()
embedding_train=np.concatenate(embedding_train,axis=0)
embedding_train.shape

(5000, 33, 4096)

In [11]:
embedding_val=df_val['most_likely'].apply(lambda x: x['embedding']).to_list()
embedding_val=np.concatenate(embedding_val,axis=0)
embedding_val.shape

(4241, 33, 4096)

In [12]:
embedding_test=df_test['most_likely'].apply(lambda x: x['embedding']).to_list()
embedding_test=np.concatenate(embedding_test,axis=0)
embedding_test.shape

(4241, 33, 4096)

In [14]:
gt_label=df_train['answer'].to_list()+df_val['answer'].to_list()+df_test['answer'].to_list()
gt_label=np.concatenate(gt_label,axis=0)
gt_label.shape

(13482,)

In [30]:
gt_label[0]

np.int64(0)

In [23]:
judge=load_judge('ScienceQA')

In [ ]:
responses=df_train['most_likely'].apply(lambda x: x['response']).to_list()+df_val['most_likely'].apply(lambda x: x['response']).to_list()+df_test['most_likely'].apply(lambda x: x['response']).to_list()

resps=[]
for x in responses:
    resps+=x
    
is_hall=[]
for i,x in enumerate(resps):
    is_hall.append(judge.check(x,gt_label[i]))

is_hall=np.array(is_hall)
print(f'There are {len(is_hall)} responses. {sum(is_hall==1)} answer correctly and {sum(is_hall==0)} answer wrongly')

There are 13482 responses. 7196 answer correctly and 6286 answer wrongly


In [9]:
len_train = len(embedding_train)
train_rate = 0.8

embedding_train_val = embedding_train
embedding_train = embedding_train_val[:int(len_train*train_rate)]
embedding_val = embedding_train_val[int(len_train*train_rate):]

embedding_test = embedding_test

y_train = is_hall[:int(len_train*train_rate)]
y_val = is_hall[int(len_train*train_rate):len_train]
y_test = is_hall[len_train:]


'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
print(f'trainset length:{len(y_train)}')
print(f'validset length:{len(y_val)}')
print(f'testset length:{len(y_test)}')
print(
    f'trainset postive: {sum(y_train==1)} negative: {sum(y_train==0)}')
print(
    f'testset postive: {sum(y_test==1)} negative: {sum(y_test==0)}')

trainset length:16000
validset length:4000
testset length:3000
trainset postive: 14127 negative: 1873
testset postive: 2625 negative: 375


In [10]:
%load_ext autoreload 
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCADiscriminator
from ml_utils.metrics import auroc,get_best_split_from_scores

Grid Search for Best 'n_components' & 'layer'

In [11]:
def evaluator(n_components,i_layer,X,y):
    discriminator=PCADiscriminator(n_components,X[:,i_layer,:])
    scores=discriminator.get_score()
    return auroc(scores,y)

In [12]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(1,12),
    'layer':range(embedding_val.shape[1])
}
grid_search=GridSearch(evaluator,grid,embedding_val,y_val)
best_paras=grid_search.search()

Grid Searching for best n_components,layer


  0%|          | 0/11 [00:00<?, ?it/s]

found best n_components:1,layer:0 Score: 0.5


/root/miniconda3/envs/dl3.9/lib/python3.9/site-packages/sklearn/decomposition/_pca.py:794: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var


found best n_components:1,layer:1 Score: 0.5032314315716201
found best n_components:1,layer:2 Score: 0.5080089104502126
found best n_components:1,layer:4 Score: 0.5090723182527264
found best n_components:1,layer:5 Score: 0.5117200169101703
found best n_components:1,layer:6 Score: 0.5160756309147798
found best n_components:1,layer:7 Score: 0.5518520253367953
found best n_components:1,layer:9 Score: 0.5604049428451658
found best n_components:1,layer:10 Score: 0.6901098627442028
found best n_components:1,layer:11 Score: 0.7616756958991651
found best n_components:1,layer:15 Score: 0.8744894189886034
found best n_components:1,layer:16 Score: 0.8903086106089381


  9%|▉         | 1/11 [00:05<00:55,  5.53s/it]/root/miniconda3/envs/dl3.9/lib/python3.9/site-packages/sklearn/decomposition/_pca.py:794: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var
 18%|█▊        | 2/11 [00:11<00:50,  5.58s/it]/root/miniconda3/envs/dl3.9/lib/python3.9/site-packages/sklearn/decomposition/_pca.py:794: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var
 27%|██▋       | 3/11 [00:16<00:45,  5.68s/it]/root/miniconda3/envs/dl3.9/lib/python3.9/site-packages/sklearn/decomposition/_pca.py:794: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var
 36%|███▋      | 4/11 [00:22<00:40,  5.73s/it]/root/miniconda3/envs/dl3.9/lib/python3.9/site-packages/sklearn/decomposition/_pca.py:794: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ra

In [13]:
best_paras

{'n_components': 1, 'layer': 16, 'best_score': np.float64(0.8903086106089381)}

In [14]:
best_layer=best_paras['layer']

View Results on Training set and Test set

In [15]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_val[:,best_layer,:])
best_split=discriminator.get_best_split(y_val)

In [16]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_train[:,best_layer,:])
scores=discriminator.get_score()
preds=(scores>best_split)
print(f'train set acc: {accuracy_score(y_train,preds)} AUROC:{roc_auc_score(y_train,scores)} AUC-PR:{aur_pr(scores,y_train)}')

train set acc: 0.8965 AUROC:0.8868287566481333 AUC-PR:0.9806494010207816


In [17]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_test[:,best_layer,:])
scores=discriminator.get_score()
preds=(scores>best_split)
print(f'test set acc: {accuracy_score(y_test,preds)} AUROC:{roc_auc_score(y_test,scores)} AUC-PR:{aur_pr(scores,y_test)}')

test set acc: 0.783 AUROC:0.8898356825396826 AUC-PR:0.9793644739734253


Train Linear Prob to Get Better Results

In [18]:
X_train=embedding_train[:, best_layer,:]
y_train=y_train

X_test=embedding_test[:, best_layer,:]
y_test=y_test

In [19]:
best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
            y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [20]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores = torch.sigmoid(output).cpu().data.numpy()
preds=scores>0.5
print(f'test set acc: {accuracy_score(y_test,preds)} AUROC:{roc_auc_score(y_test,scores)} AUC-PR:{aur_pr(scores,y_test)} f1:{f1_score(y_test,preds)}')

test set acc: 0.925 AUROC:0.9499296507936509 AUC-PR:0.9920313052733323 f1:0.9575711861210635
